# 第7章 テンプレート（Template）

## 7.1 概要

Djangoではバックエンドのテンプレートエンジンとして「DTL(Djanog Tmplate Language)」を利用することが出来る  
DTLでは特別な記法を用いて以下のことができる
- 変数表示：<br />
  変数の内容を表示する
- フィルタ：<br />
  変数の内容を表示する際の表示形式を変更する
- テンプレートタグ：<br />
  テンプレートの機能を拡張する
  
DTLには以下の特徴がある
- 「{}」「%」「|」「:」「#」を使う
- 「()」や「\[\]」は使えない
- Python風の記法
- 複雑なロジックは書けない（変数の定義などは不可）
- Pythonモジュールを直接importして使うことはできない（独自の関数を使うにはテンプレートタグやフィルタを自作する）

表示する変数は基本的に、ビュー側で事前に全て用意しておく必要がある  
筆者的には……
- ちょっとしたロジックはテンプレートタグを駆使する
- 上記でダメな場合は関数を独自テンプレートタグとして作成する

とはいえ、Djangoであらかじめ用意してあるテンプレートタグには様々な用途のものが存在するので、欲しい機能があったらまずは公式ドキュメントを見る


## 7.2 変数表示
### テンプレート内で参照できるもの
テンプレートには「コンテキスト」と呼ばれる変数と値がマッピングされたオブジェクトが渡されるため、テンプレートからはこのコンテキストに格納されている変数を参照することが出来る  
コンテキストに含まれるもの
- ビューから渡された変数（例えば、`django.shortcuts.render()`で渡されるもの）
- 設定ファイル（settings.py）の`TEMPLATES.OPTIONS.context_processors`に定義された関数でセットされた変数

Django 2.1の`context_processors`の初期設定では、次のような変数をテンプレートからデフォルトで利用できる
- `request`：<br />リクエストオブジェクト
- `user`：<br />サイトにアクセスしているユーザー
- `perms`：<br />サイトにアクセスしているユーザーのパーミッション
- `messages`：<br />フラッシュメッセージ

### テンプレート内での変数表示
変数を表示するためには、以下のような表記を用いる
~~~python
{{ <変数名> }}
~~~
例：変数`user`を表示する
~~~python
{{ user }}
~~~

~~~python
{{ user.username }}
~~~
上記のような記述をすると、以下の順で値を取得しようとして一番先に取得できた値を表示する
- `user['username']`：<br />辞書としての照合
- `user.username`：<br />属性値の照合
- `user.username()`：<br />メソッドの呼び出し
- `user[username]`：<br />リストやタプルの添字指定

#### 変数表示の応用
変数表示を応用して、`.0`や`.1`と記述することでリストの0番目、1番目を表示することが出来る  
例：変数`book_list`の0番目の`title`を表示する
~~~python
{{ book_list.0.title }}
~~~

### XSS対策とHTMLタグの出力処理
DTLで変数表示をする場合には、XSS対策としてデフォルトで`>`,`<`,`'`,`"`,`&`の文字列が自動でエスケープされる  
HTMLタグをそのまま出力したいときは、`safe`フィルタや`autoescape`タグを利用する

## 7.3 フィルタ
DTLの変数表示`{{ ～ }}`の内側にはロジックを書くことは出来ない  
代わりとして、変数の表示内容を加工するための**フィルタ**という仕組みが用意されている

### フィルタの記述
変数名の直後に`|`を使って繋げて書く 
フィルタによっては変数を取ることも出来、その場合はフィルタの直後に`:`で繋げて書く  
~~~python
{{ <変数名>|<フィルタ名> }}
{{ <変数名>|<フィルタ名>:"<引数>" }}
~~~

フィルタは`|`で次々と連結することが出来る 
`|`の前後にはスペースを入れることが出来るが、実際には入れないほうが良い  
フィルタの引数を指定するための`:`の前後には入れられない
~~~python
{{ <変数名>|<フィルタ名 1>:"<引数>"|<フィルタ名 2>:"<引数>" }}
~~~

### DTLでよく利用されるフィルタ
- `default`：<br />デフォルト表示
- `length`：<br />文字数長
- `safe`：<br />エスケープの無効化
- `date`：<br />日時フォーマット
- `linebreaksbr`：<br />改行タグの変換
- `urlize`：<br />リンクの変換
- `truncatechars_html`：<br />文字の切り詰め

### 7.3.1 `default`（デフォルト表示）
変数が存在しない場合、あるいは変数の値が`None`,`''`,`0`,`False`,`[]`等の場合に、指定した文字列を表示することが出来る  
特に、値が`None`の場合に「None」という文字列がそのまま出てしまうのを回避するためによく利用する
~~~python
{{ user.first_name|default:"" }} {{ user.last_name:default:"" }}
~~~

#### `default_if_none`
値が`None`のときだけ指定した文字列に変換する

### 7.3.2 `length`（文字列長）
変数の値の文字列長を表示する
~~~python
{{ user.username|length }}
~~~

### 7.3.3 `safe`（エスケープの無効化）
変数表示の際のXSS対策を無効化する  
表示する変数の内容が安全だと分かっているときのみ使用する  
~~~python
{{ book.description|safe }}
~~~

### 7.3.4 `date`（日時フォーマット）
日時オブジェクトを引数に指定した形式に変換して表示する
~~~python
{{ user.last_login|date:"Y-m-d H:i:s" }}
~~~
と記述すると、次のようにレンダリングされる
> 2018-04-22 10:22:30

### 7.3.5 `linebreaksbr`（改行タグの変換）
`\n`を`<br>`に変換する  
テキストエリアに`\n`が含まれる文字列を表示するときによく使われる
~~~python
{{ book.descripstion|linebreaksbr }}
~~~

### 7.3.6 `urlize`（リンク変換）
文字列内にURLとメールアドレスが含まれる場合に、その部分だけをアンカータグで囲んでクリック可能なリンクに変換する
例えば、`book.description`の内容が「`http://akiyoko.hatenablog.jp/`も見てね。」であれば、
~~~python
{{ book.description|urlize }}
~~~
と記述すると、次のように変換して表示される
> `<a href="http://akiyoko.hatenablog.jp/">http://akiyoko.hatenablog.jp/</a>も見てね。`

### 7.3.7 truncatechars_html（文字の切り詰め）
文字列が長い場合に、決められた文字数まで切り詰めた後で「...」という文字を付与する  
`truncatechars_html`だと、HTMLタグを考慮して省略後の文字に切り詰めてくれる  
例えば、`book.description`の内容が「`<b>絶賛頒布中です。</b>`」であれば、
~~~python
{{ book.description|truncatechars_html:5 }}
~~~
と記述すると、次のように変換して表示される
> `<b>絶賛...</b>`